This file is about trialling LSA as an approximator of loss. 


In [20]:
#build a set of LSA compliant tensors
import torch
B=30
Base=torch.arange(0, 40, 1, dtype=torch.float32)
#repeat the base tensor 10 times
Base=Base.unsqueeze(-1).repeat(1,B)
#shuffle the base tensor
Base=Base[torch.randperm(Base.size()[0])]
#one hot encode the base tensor (100x100xB)
Base=torch.nn.functional.one_hot(Base.to(torch.int64), num_classes=40).permute(1,0,2)
print(Base.shape)
print(Base[0])


torch.Size([30, 40, 40])
tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 1, 0,  ..., 0, 0, 0]])


In [33]:
Loss=torch.nn.CrossEntropyLoss(reduction="none")
Base=Base.float().to("cuda")
#profile how long it takes to compute the loss

with torch.autograd.profiler.profile(use_cuda=True, profile_memory=True) as prof: 
    tgt=torch.arange(Base.shape[-1], dtype=torch.int64,device=Base.device).unsqueeze(0).repeat(Base.shape[0],1)
    loss=Loss(Base, tgt)

print(prof.key_averages().table(sort_by="cuda_time_total"))
print(loss.shape)


-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg       CPU Mem  Self CPU Mem      CUDA Mem  Self CUDA Mem    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                               aten::cross_entropy_loss         4.06%      30.000us        48.04%     355.000us     355.000us      26.000us         3.18%     361.000us     361.000us           0 b           0 b       5.00 Kb    -187.50 K

STAGE:2023-10-03 13:27:25 9428:9428 ActivityProfilerController.cpp:311] Completed Stage: Warm Up
STAGE:2023-10-03 13:27:25 9428:9428 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-10-03 13:27:25 9428:9428 ActivityProfilerController.cpp:321] Completed Stage: Post Processing
